# Week 28 (9th July - 15th July 2018) - Number of New Ads

In [1]:
#packages
import pandas as pd
import csv
import matplotlib as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from statsmodels.formula.api import ols
import glob
import pycountry
from datetime import datetime

#chaning the number of columns that are displayed. get will tell you how much. 
pd.set_option("display.max_columns",60)
pd.set_option("display.max_rows",1000)
sns.set_style("whitegrid")

#currency converter
from currency_converter import CurrencyConverter
c = CurrencyConverter('http://www.ecb.int/stats/eurofxref/eurofxref-hist.zip')

pd.options.mode.chained_assignment = None

cc={"total_y": "New Ads W27", "margin %_y": "Margin % W27", "total_x": "New Ads W28",
                                      "margin %_x": "Margin % W28"}

In [2]:
##### Cleaning

#1. Import most recent week
df_new = pd.read_excel("28_newads.xlsx")
df_prev = pd.read_excel("27_newads.xlsx")

#2.Create function to number new ads for both weeks
df_new["total"] = 0
df_prev["total"] = 0
def bim(x):
    if x["insert_time"] >= datetime(2018,7,9):
        return(x["total"] + 1)
    else:
        return(x["total"])
def bam(x):
    if x["insert_time"] >= datetime(2018,7,2):
        return(x["total"] + 1)
    else:
        return(x["total"])    
df_new["total"] = df_new.apply(bim, axis = 1)
df_prev["total"] = df_prev.apply(bam, axis = 1)

#3. Create Margin % function
def woo(x):
    if x["purchase"] == 0 and x["revenue"] >= 1:
        return(100)
    if x["purchase"] == 0 and x["revenue"] == 0:
        return(0)
    if x["purchase"] > 0 and x["revenue"] == 0:
        return(-100)
    else:
        return((x["margin"]/x["revenue"])*100)

#4. Drop crap
df_prev.drop(columns = ["id","status","real_budget","start_date","end_date","url","ad_set_campaign_id",
                          "margin_percentage","facebook_ad_set_id","facebook_ad_account_id","facebook_business_id",
                          "short_url","ad_url_domain_id"], inplace=True)
df_new.drop(columns = ["id","status","real_budget","start_date","end_date","url","ad_set_campaign_id",
                          "margin_percentage","facebook_ad_set_id","facebook_ad_account_id","facebook_business_id",
                          "short_url","ad_url_domain_id"], inplace=True)

# Other

## Jeanine de Konnigh

The Twin Promotion campaigns could proably use some maintenance. Most of the campaigns which decreased in margin are inactive, so they should be ok!

In [3]:
df_jeanine1=df_new.loc[df_new["marketer_name"]=="Jeanine de Konnigh"]
df_jeanine1=df_jeanine1.groupby(by=['campaign_name']).sum()
df_jeanine1["margin %"] = df_jeanine1.apply(woo, axis=1)
df_jeanine1["margin %"] = df_jeanine1["margin %"].round(2)

df_jeanine2=df_prev.loc[df_prev["marketer_name"]=="Jeanine de Konnigh"]
df_jeanine2=df_jeanine2.groupby(by=['campaign_name']).sum()
df_jeanine2["margin %"] = df_jeanine2.apply(woo, axis=1)
df_jeanine2["margin %"] = df_jeanine2["margin %"].round(2)

df_jeanine = df_jeanine1.merge(df_jeanine2, how ="outer", on = ["campaign_name"])
df_jeanine = df_jeanine[["total_y","margin %_y","total_x","margin %_x"]]
df_jeanine = df_jeanine.rename(index=str, columns=cc)

df_jeanine["Margin % Difference from Previous Week"]=df_jeanine["Margin % W28"]-(df_jeanine["Margin % W27"])

df_jeanine

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Afrika Museum Like,5.0,32.05,0.0,29.48,-2.57
Compare & Connect - E-Bike AU UD,NaN,NaN,6.0,31.03,NaN
Compare & Connect - Voucher LEAD ADS - KitchenAid,0.0,55.61,0.0,59.30,3.69
Compare & Connect - Voucher LEAD ADS - eBike,NaN,NaN,0.0,-100.00,NaN
DFDS NL - Schotland,NaN,NaN,4.0,76.21,NaN
Divina Pastora - Dental Insurance - Lead Ads,2.0,42.73,0.0,51.05,8.32
Divina Pastora - Seguro Mascotas Lead Ads,2.0,44.59,0.0,49.49,4.90
Intereno NL - RSP,NaN,NaN,0.0,100.00,NaN
Intereno NL - RSP A/B test,2.0,60.64,2.0,61.49,0.85


## Onno Bossen

The new Prescan NL Budget Optimization ads seem to have reduced the margin a fair bit. I assume this was just a test though? Care Vision AT - LA - Info pack could use some maintenance. 

In [4]:
df_onno1=df_new.loc[df_new["marketer_name"]=="Onno Bossen"]
df_onno1=df_onno1.groupby(by=['campaign_name']).sum()
df_onno1["margin %"] = df_onno1.apply(woo, axis=1)
df_onno1["margin %"] = df_onno1["margin %"].round(2)

df_onno2=df_prev.loc[df_prev["marketer_name"]=="Onno Bossen"]
df_onno2=df_onno2.groupby(by=['campaign_name']).sum()
df_onno2["margin %"] = df_onno2.apply(woo, axis=1)
df_onno2["margin %"] = df_onno2["margin %"].round(2)

df_onno = df_onno1.merge(df_onno2, how ="outer", on = ["campaign_name"])
df_onno = df_onno[["total_y","margin %_y","total_x","margin %_x"]]
df_onno = df_onno.rename(index=str, columns=cc)

df_onno["Margin % Difference from Previous Week"]=df_onno["Margin % W28"]-(df_onno["Margin % W27"])

df_onno

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Care Laser NL - lead ads,2.0,64.16,0,69.37,5.21
Care Vision AT - LA - Info pack,0.0,86.96,0,57.31,-29.65
Care Vision AT - LP,0.0,42.21,0,30.18,-12.03
Care Vision DE - LA - Info pack,0.0,51.83,0,35.43,-16.40
Care Vision DE - LP,2.0,-86.34,0,8.66,95.00
Medicare Angola NA PT,0.0,56.13,0,53.71,-2.42
Medicare Dental Testers PT NA,0.0,38.13,0,41.62,3.49
Medicare Familia PT NA,0.0,20.94,0,32.66,11.72
Medicare Familia PT NA - V2,0.0,22.74,0,28.68,5.94


________________

# NGOs

## Abby Listerman

Proably best to wait and see how successful the new ads were after this week. 

In [5]:
df_abby1=df_new.loc[df_new["marketer_name"]=="Abby Listerman"]
df_abby1=df_abby1.groupby(by=['campaign_name']).sum()
df_abby1["margin %"] = df_abby1.apply(woo, axis=1)
df_abby1["margin %"] = df_abby1["margin %"].round(2)

df_abby2=df_prev.loc[df_prev["marketer_name"]=="Abby Listerman"]
df_abby2=df_abby2.groupby(by=['campaign_name']).sum()
df_abby2["margin %"] = df_abby2.apply(woo, axis=1)
df_abby2["margin %"] = df_abby2["margin %"].round(2)

df_abby = df_abby1.merge(df_abby2, how ="outer", on = ["campaign_name"])
df_abby = df_abby[["total_y","margin %_y","total_x","margin %_x"]]
df_abby = df_abby.rename(index=str, columns=cc)

df_abby["Margin % Difference from Previous Week"]=df_abby["Margin % W28"]-(df_abby["Margin % W27"])

df_abby

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Alannah and Madeline Foundation AU,0.0,70.41,1.0,63.48,-6.93
Greenpeace MX - Pipelines - Lead Ads,NaN,NaN,4.0,89.19,NaN
Greenpeace MX - Plastics - Lead ads,NaN,NaN,0.0,85.35,NaN
SPCA NZ,1.0,67.35,0.0,57.55,-9.80
UNHCR Australia,0.0,35.48,3.0,32.34,-3.14
WWF Germany - Against Coal,0.0,57.29,3.0,52.49,-4.80
CICC,0.0,60.37,NaN,NaN,NaN
Coastguard New Zealand - Quiz,0.0,58.89,NaN,NaN,NaN
Greenpeace Taiwan - Antarctic,2.0,25.37,NaN,NaN,NaN


## Mimmi Ångström

Looks fine for now!

In [6]:
df_mimmi1=df_new.loc[df_new["marketer_name"]=="Mimmi Ångström"]
df_mimmi1=df_mimmi1.groupby(by=['campaign_name']).sum()
df_mimmi1["margin %"] = df_mimmi1.apply(woo, axis=1)
df_mimmi1["margin %"] = df_mimmi1["margin %"].round(2)

df_mimmi2=df_prev.loc[df_prev["marketer_name"]=="Mimmi Ångström"]
df_mimmi2=df_mimmi2.groupby(by=['campaign_name']).sum()
df_mimmi2["margin %"] = df_mimmi2.apply(woo, axis=1)
df_mimmi2["margin %"] = df_mimmi2["margin %"].round(2)

df_mimmi = df_mimmi1.merge(df_mimmi2, how ="outer", on = ["campaign_name"])
df_mimmi = df_mimmi[["total_y","margin %_y","total_x","margin %_x"]]
df_mimmi = df_mimmi.rename(index=str, columns=cc)

df_mimmi["Margin % Difference from Previous Week"]=df_mimmi["Margin % W28"]-(df_mimmi["Margin % W27"])

df_mimmi

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Aldeas Infantiles SOS Brasil - Sexual Abuse - Lead ads,0,85.49,0.0,83.12,-2.37
Centrepoint UK,5,71.70,0.0,77.89,6.19
UNICEF Turkish National Committee - Yemen - Lead Ads,2,94.51,0.0,92.10,-2.41
WDC Whales and Dolphins - UK,0,79.64,0.0,73.61,-6.03
Barnfonden - FGM,0,75.60,NaN,NaN,NaN
CareFlight - Petition - Lead Ads,0,42.39,NaN,NaN,NaN
CareFlight - Quiz - About CareFlight,0,66.30,NaN,NaN,NaN
Make A Wish Australia,0,52.16,NaN,NaN,NaN
SOS Children's Villages India - Lead Ads,0,76.66,NaN,NaN,NaN


## Marleen Blokker

Since most campaigns were out of action the week before, there is nothing to comment on.

In [7]:
df_marleen1=df_new.loc[df_new["marketer_name"]=="Marleen Blokker"]
df_marleen1=df_marleen1.groupby(by=['campaign_name']).sum()
df_marleen1["margin %"] = df_marleen1.apply(woo, axis=1)
df_marleen1["margin %"] = df_marleen1["margin %"].round(2)

df_marleen2=df_prev.loc[df_prev["marketer_name"]=="Marleen Blokker"]
df_marleen2=df_marleen2.groupby(by=['campaign_name']).sum()
df_marleen2["margin %"] = df_marleen2.apply(woo, axis=1)
df_marleen2["margin %"] = df_marleen2["margin %"].round(2)

df_marleen = df_marleen1.merge(df_marleen2, how ="outer", on = ["campaign_name"])
df_marleen = df_marleen[["total_y","margin %_y","total_x","margin %_x"]]
df_marleen = df_marleen.rename(index=str, columns=cc)

df_marleen["Margin % Difference from Previous Week"]=df_marleen["Margin % W28"]-(df_marleen["Margin % W27"])

df_marleen

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
CICC,NaN,NaN,3.0,57.22,NaN
CareFlight - Petition - Lead Ads,NaN,NaN,0.0,43.98,NaN
CareFlight - Quiz - About CareFlight,NaN,NaN,0.0,60.09,NaN
CareFlight - Quiz - First Aid,NaN,NaN,0.0,68.45,NaN
Coastguard New Zealand - Quiz,NaN,NaN,0.0,54.63,NaN
Greenpeace NL - Wereld zonder plastic,NaN,NaN,0.0,42.11,NaN
Greenpeace Taiwan - Antarctic,NaN,NaN,1.0,42.57,NaN
Greenpeace Taiwan - Antarctic - lead ads,NaN,NaN,3.0,50.38,NaN
MSF AU - Rohingya,NaN,NaN,0.0,-100.00,NaN


## Daniela Jacobo Gallardo

Really good city ads for Aldeas Infantiles SOS ES!

In [8]:
df_daniela1=df_new.loc[df_new["marketer_name"]=="Daniela Jacobo Gallardo"]
df_daniela1=df_daniela1.groupby(by=['campaign_name']).sum()
df_daniela1["margin %"] = df_daniela1.apply(woo, axis=1)
df_daniela1["margin %"] = df_daniela1["margin %"].round(2)

df_daniela2=df_prev.loc[df_prev["marketer_name"]=="Daniela Jacobo Gallardo"]
df_daniela2=df_daniela2.groupby(by=['campaign_name']).sum()
df_daniela2["margin %"] = df_daniela2.apply(woo, axis=1)
df_daniela2["margin %"] = df_daniela2["margin %"].round(2)

df_daniela = df_daniela1.merge(df_daniela2, how ="outer", on = ["campaign_name"])
df_daniela = df_daniela[["total_y","margin %_y","total_x","margin %_x"]]
df_daniela = df_daniela.rename(index=str, columns=cc)

df_daniela["Margin % Difference from Previous Week"]=df_daniela["Margin % W28"]-(df_daniela["Margin % W27"])

df_daniela

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Aldeas Infantiles SOS ES - Child Violence V3,3.0,8.76,0.0,8.37,-0.39
Aldeas Infantiles SOS ES - Hermanos - Lead Ads,0.0,37.97,0.0,30.66,-7.31
Aldeas Infantiles SOS de Espana - Bullying - City ads,12.0,39.21,127.0,69.66,30.45
Aldeas Infantiles SOS de Espana - Bullying - Lead Ads,2.0,30.41,0.0,27.91,-2.50
Aldeas SOS ES - Spain Poverty V3,NaN,NaN,4.0,-124.22,NaN
Ayuda en Accion de Mexico - Violence Against Women,0.0,75.77,0.0,77.84,2.07
Ayuda en Acción - Human Trafficking lead ads,0.0,54.64,0.0,55.68,1.04
Ayuda en Acción - Mexican poverty - Lead ads,0.0,75.23,0.0,74.77,-0.46
Barnardos NZ,0.0,41.38,0.0,16.45,-24.93


## Sam Drentje

In [9]:
df_sam1=df_new.loc[df_new["marketer_name"]=="Sam Drentje"]
df_sam1=df_sam1.groupby(by=['campaign_name']).sum()
df_sam1["margin %"] = df_sam1.apply(woo, axis=1)
df_sam1["margin %"] = df_sam1["margin %"].round(2)

df_sam2=df_prev.loc[df_prev["marketer_name"]=="Sam Drentje"]
df_sam2=df_sam2.groupby(by=['campaign_name']).sum()
df_sam2["margin %"] = df_sam2.apply(woo, axis=1)
df_sam2["margin %"] = df_sam2["margin %"].round(2)

df_sam = df_sam1.merge(df_sam2, how ="outer", on = ["campaign_name"])
df_sam = df_sam[["total_y","margin %_y","total_x","margin %_x"]]
df_sam = df_sam.rename(index=str, columns=cc)

df_sam["Margin % Difference from Previous Week"]=df_sam["Margin % W28"]-(df_sam["Margin % W27"])

df_sam

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
ChildFund New Zealand - Violence - Lead Ads,0,44.61,0.0,22.27,-22.34
Childfund New Zealand - Forced Marriage - Lead ads,0,14.03,0.0,32.48,18.45
Stichting DierenLot - steunbetuiging - Lead Ads,0,31.09,0.0,33.47,2.38
Stichting DierenLot - steunbetuiging ambulance - Lead Ads,0,29.07,0.0,29.96,0.89
CARE France - Violence against women LP,0,100.00,NaN,NaN,NaN
Stichting DierenLot - ambulance,0,-100.00,NaN,NaN,NaN


_______________

# Hearing Aids

## Rob Ransome

Most seem to be ok, I would watch out for Nessa Hearing MY - Lead ads ENG though.

In [10]:
df_rob1=df_new.loc[df_new["marketer_name"]=="Rob Ransome"]
df_rob1=df_rob1.groupby(by=['campaign_name']).sum()
df_rob1["margin %"] = df_rob1.apply(woo, axis=1)
df_rob1["margin %"] = df_rob1["margin %"].round(2)

df_rob2=df_prev.loc[df_prev["marketer_name"]=="Rob Ransome"]
df_rob2=df_rob2.groupby(by=['campaign_name']).sum()
df_rob2["margin %"] = df_rob2.apply(woo, axis=1)
df_rob2["margin %"] = df_rob2["margin %"].round(2)

df_rob = df_rob1.merge(df_rob2, how ="outer", on = ["campaign_name"])
df_rob = df_rob[["total_y","margin %_y","total_x","margin %_x"]]
df_rob = df_rob.rename(index=str, columns=cc)

df_rob["Margin % Difference from Previous Week"]=df_rob["Margin % W28"]-(df_rob["Margin % W27"])

df_rob

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Advanced Hearing,0.0,65.64,0.0,28.34,-37.30
Belsono - Belsono Hearing Centers - Experience/Demo,0.0,29.19,4.0,35.06,5.87
Belsono - Belsono Hearing Centers - Flexible City Ads,0.0,69.46,0.0,-100.00,-169.46
Belsono - Hearing Health - Experience/Demo,0.0,36.08,0.0,-11.93,-48.01
Belsono - Hearing Health - Flexible City ads,NaN,NaN,0.0,100.00,NaN
Belsono - Hearing Health - Free Evaluation,0.0,43.50,0.0,46.68,3.18
Belsono Hearing Health - USA - Muse iQR - TW - RSP - City,67.0,73.78,48.0,58.96,-14.82
Belsono Hearing Health - USA - Muse iQR - TW - RSP - Flexible Cities,26.0,52.08,0.0,63.55,11.47
Belsono Hearing Health - USA - Muse iQR - TW - RSP - Reg,4.0,58.50,5.0,56.48,-2.02


## Robin Ten Hove

Looks good

In [11]:
df_robin1=df_new.loc[df_new["marketer_name"]=="Robin Ten Hove"]
df_robin1=df_robin1.groupby(by=['campaign_name']).sum()
df_robin1["margin %"] = df_robin1.apply(woo, axis=1)
df_robin1["margin %"] = df_robin1["margin %"].round(2)

df_robin2=df_prev.loc[df_prev["marketer_name"]=="Robin Ten Hove"]
df_robin2=df_robin2.groupby(by=['campaign_name']).sum()
df_robin2["margin %"] = df_robin2.apply(woo, axis=1)
df_robin2["margin %"] = df_robin2["margin %"].round(2)

df_robin = df_robin1.merge(df_robin2, how ="outer", on = ["campaign_name"])
df_robin = df_robin[["total_y","margin %_y","total_x","margin %_x"]]
df_robin = df_robin.rename(index=str, columns=cc)

df_robin["Margin % Difference from Previous Week"]=df_robin["Margin % W28"]-(df_robin["Margin % W27"])

df_robin

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
AudioNova BEFR - White label IIC,0.0,55.61,0.0,50.51,-5.10
AudioNova BENL - QZ V3 IIC - General,0.0,16.99,0.0,22.46,5.47
AudioNova IT - Luna C - RSP - Regular,0.0,28.21,0.0,29.88,1.67
Audionova IT - Luna C - QZ V5,NaN,NaN,4.0,28.67,NaN
GEERS PL,0.0,100.00,0.0,100.00,0.00
GEERS PL - Luna R-R - QZ,0.0,46.30,3.0,52.81,6.51
GEERS PL - Luna R-R - QZ - Cities,0.0,18.54,0.0,55.51,36.97
GEERS PL - QZ V1,0.0,52.34,0.0,52.67,0.33
HelloFresh BEFR - Hoger lager KitchenAid,0.0,100.00,0.0,100.00,0.00


## Philip Snoey Kiwiet

Consider new ads fror Akoustica Medica

In [12]:
df_philip1=df_new.loc[df_new["marketer_name"]=="Philip Snoey Kiwiet"]
df_philip1=df_philip1.groupby(by=['campaign_name']).sum()
df_philip1["margin %"] = df_philip1.apply(woo, axis=1)
df_philip1["margin %"] = df_philip1["margin %"].round(2)

df_philip2=df_prev.loc[df_prev["marketer_name"]=="Philip Snoey Kiwiet"]
df_philip2=df_philip2.groupby(by=['campaign_name']).sum()
df_philip2["margin %"] = df_philip2.apply(woo, axis=1)
df_philip2["margin %"] = df_philip2["margin %"].round(2)

df_philip = df_philip1.merge(df_philip2, how ="outer", on = ["campaign_name"])
df_philip = df_philip[["total_y","margin %_y","total_x","margin %_x"]]
df_philip = df_philip.rename(index=str, columns=cc)

df_philip["Margin % Difference from Previous Week"]=df_philip["Margin % W28"]-(df_philip["Margin % W27"])

df_philip

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Akoustica Medica,0,62.04,0.0,52.7,-9.34
Akoustica Medica - Split test,3,50.65,NaN,NaN,NaN


## Theresa Majic

New ads seem to be working well. Since most new ads were only added during 28, best to wait and see.  

In [13]:
df_theresa1=df_new.loc[df_new["marketer_name"]=="Theresa Majic"]
df_theresa1=df_theresa1.groupby(by=['campaign_name']).sum()
df_theresa1["margin %"] = df_theresa1.apply(woo, axis=1)
df_theresa1["margin %"] = df_theresa1["margin %"].round(2)

df_theresa2=df_prev.loc[df_prev["marketer_name"]=="Theresa Majic"]
df_theresa2=df_theresa2.groupby(by=['campaign_name']).sum()
df_theresa2["margin %"] = df_theresa2.apply(woo, axis=1)
df_theresa2["margin %"] = df_theresa2["margin %"].round(2)

df_theresa = df_theresa1.merge(df_theresa2, how ="outer", on = ["campaign_name"])
df_theresa = df_theresa[["total_y","margin %_y","total_x","margin %_x"]]
df_theresa = df_theresa.rename(index=str, columns=cc)

df_theresa["Margin % Difference from Previous Week"]=df_theresa["Margin % W28"]-(df_theresa["Margin % W27"])

df_theresa

,New Ads W27,Margin % W27,New Ads W28,Margin % W28,Margin % Difference from Previous Week
campaign_name,,,,,
Amplifon BEFR - Signia Cellion,0.0,62.02,0.0,55.16,-6.86
Amplifon BENL - Signia Cellion,0.0,33.40,4.0,34.73,1.33
Amplifon DE - ReSound LiNX 3D,0.0,60.10,0.0,62.77,2.67
Amplifon ES - ReSound LiNX 3D,0.0,92.03,4.0,90.61,-1.42
Amplifon Switzerland,0.0,62.95,0.0,74.22,11.27
AudicaoActiva PT - MIC - Lead Ads - New,0.0,19.74,8.0,31.09,11.35
AudicaoActiva PT - ReSound - Lead Ads - New,3.0,31.07,3.0,39.43,8.36
Audio Clinic MX,NaN,NaN,4.0,97.15,NaN
AudioNova DK - Moxi Now QZ - Cities,NaN,NaN,0.0,44.51,NaN


In [14]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>